# Predicting House Sale Prices

In this project, we use the Machine Learning Algorithm Linear Regression to predict the house sale prices of the city of Ames, Iowa, United States from 2006 to 2010.

Let us first import some necessary libraries and then read the data file into a data frame.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

In [2]:
df=pd.read_csv('AmesHousing.tsv', delimiter="\t")
df

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,2926,923275080,80,RL,37.0,7937,Pave,NaN,IR1,Lvl,...,0,NaN,GdPrv,NaN,0,3,2006,WD,Normal,142500
2926,2927,923276100,20,RL,NaN,8885,Pave,NaN,IR1,Low,...,0,NaN,MnPrv,NaN,0,6,2006,WD,Normal,131000
2927,2928,923400125,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal,132000
2928,2929,924100070,20,RL,77.0,10010,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2006,WD,Normal,170000


Before going into details of the algorithm, one need to perform features enginnering and features selection.

# Features Enginneering

What we need to do is to detect and remove features with many missing values, discover potential categorical features and then transform text and numerical columns. 

First of all, we remove all the columns having more than or equal to 5% of missing values.  

In [3]:
missing_sum=df.isnull().sum()
print(missing_sum)
print('\n')

missing_columns=missing_sum[missing_sum/len(df) >=0.05]
print(missing_columns)

df=df.drop(missing_columns.index, axis=1)
df

Order               0
PID                 0
MS SubClass         0
MS Zoning           0
Lot Frontage      490
                 ... 
Mo Sold             0
Yr Sold             0
Sale Type           0
Sale Condition      0
SalePrice           0
Length: 82, dtype: int64


Lot Frontage      490
Alley            2732
Fireplace Qu     1422
Garage Type       157
Garage Yr Blt     159
Garage Finish     159
Garage Qual       159
Garage Cond       159
Pool QC          2917
Fence            2358
Misc Feature     2824
dtype: int64


,Order,PID,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,...,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,31770,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,11622,Pave,Reg,Lvl,AllPub,Inside,...,0,0,120,0,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,14267,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,11160,Pave,Reg,Lvl,AllPub,Corner,...,0,0,0,0,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,13830,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,3,2010,WD,Normal,189900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,2926,923275080,80,RL,7937,Pave,IR1,Lvl,AllPub,CulDSac,...,0,0,0,0,0,3,2006,WD,Normal,142500
2926,2927,923276100,20,RL,8885,Pave,IR1,Low,AllPub,Inside,...,0,0,0,0,0,6,2006,WD,Normal,131000
2927,2928,923400125,85,RL,10441,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,700,7,2006,WD,Normal,132000
2928,2929,924100070,20,RL,10010,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,4,2006,WD,Normal,170000


For the other numeric columns, which contains less than 5% of missing values, one can replace these values by the most common value of that column, i.e., the mode.  

In [4]:
number_numeric_missing_columns=df.select_dtypes(include = ['int', 'float']).isnull().sum()
numeric_missing_columns=number_numeric_missing_columns[number_numeric_missing_columns>0] # we consider the columns having at least 1 missing value
print(numeric_missing_columns)
print('\n')

common_values=df[numeric_missing_columns.index].mode()
common_values



Mas Vnr Area      23
BsmtFin SF 1       1
BsmtFin SF 2       1
Bsmt Unf SF        1
Total Bsmt SF      1
Bsmt Full Bath     2
Bsmt Half Bath     2
Garage Cars        1
Garage Area        1
dtype: int64




,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Bsmt Full Bath,Bsmt Half Bath,Garage Cars,Garage Area
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0


In order to fill the missing values of our data frame, one need to convert these mode data frame to dictionary.

In [5]:
common_values_dict=common_values.to_dict(orient='records')[0]
print(common_values_dict)
print('\n')

df = df.fillna(common_values_dict)
df

{'Mas Vnr Area': 0.0, 'BsmtFin SF 1': 0.0, 'BsmtFin SF 2': 0.0, 'Bsmt Unf SF': 0.0, 'Total Bsmt SF': 0.0, 'Bsmt Full Bath': 0.0, 'Bsmt Half Bath': 0.0, 'Garage Cars': 2.0, 'Garage Area': 0.0}




,Order,PID,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,...,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,31770,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,11622,Pave,Reg,Lvl,AllPub,Inside,...,0,0,120,0,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,14267,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,11160,Pave,Reg,Lvl,AllPub,Corner,...,0,0,0,0,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,13830,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,3,2010,WD,Normal,189900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,2926,923275080,80,RL,7937,Pave,IR1,Lvl,AllPub,CulDSac,...,0,0,0,0,0,3,2006,WD,Normal,142500
2926,2927,923276100,20,RL,8885,Pave,IR1,Low,AllPub,Inside,...,0,0,0,0,0,6,2006,WD,Normal,131000
2927,2928,923400125,85,RL,10441,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,700,7,2006,WD,Normal,132000
2928,2929,924100070,20,RL,10010,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,4,2006,WD,Normal,170000


How about non-numeric columns? I.e., text columns. We just delete the columns having a missing value.

In [6]:
text_counts = df.select_dtypes(include=['object']).isnull().sum()

missing_text_cols = text_counts[text_counts > 0]

df = df.drop(missing_text_cols.index, axis=1)
df

,Order,PID,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,...,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,31770,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,11622,Pave,Reg,Lvl,AllPub,Inside,...,0,0,120,0,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,14267,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,11160,Pave,Reg,Lvl,AllPub,Corner,...,0,0,0,0,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,13830,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,3,2010,WD,Normal,189900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,2926,923275080,80,RL,7937,Pave,IR1,Lvl,AllPub,CulDSac,...,0,0,0,0,0,3,2006,WD,Normal,142500
2926,2927,923276100,20,RL,8885,Pave,IR1,Low,AllPub,Inside,...,0,0,0,0,0,6,2006,WD,Normal,131000
2927,2928,923400125,85,RL,10441,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,700,7,2006,WD,Normal,132000
2928,2929,924100070,20,RL,10010,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,4,2006,WD,Normal,170000


It looks fine for now. Let us check again if we have cleaned all the missing values in our data frame.

In [7]:
df.isnull().sum().value_counts()

0    64
dtype: int64

Next step is to creat new useful features from existing ones. Observe that there are two columns 'Yr Sold' and 'Year Build', they do not provide too much necessary information. But the diffrence between them is actually a very interesting information since it gives us the age of the house.

Similarly, one can do the same thing with the column 'Year Remod/add' indicating the year when the house was remodeled. Keep in mind that there are some cases in which this difference is negative, i.e, something wrong with the data. In that case, we just need to delete them.

In [8]:
df['Year_since_built']=df['Yr Sold'] - df['Year Built']
print(df['Year_since_built'][df['Year_since_built']<0])
print('\n')

df['Year_since_remod']=df['Yr Sold'] - df['Year Remod/Add']
print(df['Year_since_remod'][df['Year_since_remod']<0])

df=df.drop([1702,2180,2181],axis=0)
df


2180   -1
Name: Year_since_built, dtype: int64


1702   -1
2180   -2
2181   -1
Name: Year_since_remod, dtype: int64


,Order,PID,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,...,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice,Year_since_built,Year_since_remod
0,1,526301100,20,RL,31770,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,5,2010,WD,Normal,215000,50,50
1,2,526350040,20,RH,11622,Pave,Reg,Lvl,AllPub,Inside,...,120,0,0,6,2010,WD,Normal,105000,49,49
2,3,526351010,20,RL,14267,Pave,IR1,Lvl,AllPub,Corner,...,0,0,12500,6,2010,WD,Normal,172000,52,52
3,4,526353030,20,RL,11160,Pave,Reg,Lvl,AllPub,Corner,...,0,0,0,4,2010,WD,Normal,244000,42,42
4,5,527105010,60,RL,13830,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,3,2010,WD,Normal,189900,13,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,2926,923275080,80,RL,7937,Pave,IR1,Lvl,AllPub,CulDSac,...,0,0,0,3,2006,WD,Normal,142500,22,22
2926,2927,923276100,20,RL,8885,Pave,IR1,Low,AllPub,Inside,...,0,0,0,6,2006,WD,Normal,131000,23,23
2927,2928,923400125,85,RL,10441,Pave,Reg,Lvl,AllPub,Inside,...,0,0,700,7,2006,WD,Normal,132000,14,14
2928,2929,924100070,20,RL,10010,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,4,2006,WD,Normal,170000,32,31


And yes, the old columns 'Year Built' and 'Year Remod/Add' should then be removed.

In [9]:
df=df.drop(['Year Built' , 'Year Remod/Add'], axis=1)
df

,Order,PID,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,...,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice,Year_since_built,Year_since_remod
0,1,526301100,20,RL,31770,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,5,2010,WD,Normal,215000,50,50
1,2,526350040,20,RH,11622,Pave,Reg,Lvl,AllPub,Inside,...,120,0,0,6,2010,WD,Normal,105000,49,49
2,3,526351010,20,RL,14267,Pave,IR1,Lvl,AllPub,Corner,...,0,0,12500,6,2010,WD,Normal,172000,52,52
3,4,526353030,20,RL,11160,Pave,Reg,Lvl,AllPub,Corner,...,0,0,0,4,2010,WD,Normal,244000,42,42
4,5,527105010,60,RL,13830,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,3,2010,WD,Normal,189900,13,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,2926,923275080,80,RL,7937,Pave,IR1,Lvl,AllPub,CulDSac,...,0,0,0,3,2006,WD,Normal,142500,22,22
2926,2927,923276100,20,RL,8885,Pave,IR1,Low,AllPub,Inside,...,0,0,0,6,2006,WD,Normal,131000,23,23
2927,2928,923400125,85,RL,10441,Pave,Reg,Lvl,AllPub,Inside,...,0,0,700,7,2006,WD,Normal,132000,14,14
2928,2929,924100070,20,RL,10010,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,4,2006,WD,Normal,170000,32,31


Next, we need to remove the columns that are not useful. There are two types of them. First of all, the columns 'Order' and 'PID' are numeric but they do not contain any useful information. The second, the columns 'Mo Sold', 'Sale Condition', 'Sale Type', 'Yr Sold' leak the information about the final sale. 

In [10]:
df=df.drop(['Order', 'PID', 'Mo Sold', 'Sale Condition', 'Sale Type', 'Yr Sold'], axis=1)
df

,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,...,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,SalePrice,Year_since_built,Year_since_remod
0,20,RL,31770,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,...,210,62,0,0,0,0,0,215000,50,50
1,20,RH,11622,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,...,140,0,0,0,120,0,0,105000,49,49
2,20,RL,14267,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,...,393,36,0,0,0,0,12500,172000,52,52
3,20,RL,11160,Pave,Reg,Lvl,AllPub,Corner,Gtl,NAmes,...,0,0,0,0,0,0,0,244000,42,42
4,60,RL,13830,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,...,212,34,0,0,0,0,0,189900,13,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,80,RL,7937,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,Mitchel,...,120,0,0,0,0,0,0,142500,22,22
2926,20,RL,8885,Pave,IR1,Low,AllPub,Inside,Mod,Mitchel,...,164,0,0,0,0,0,0,131000,23,23
2927,85,RL,10441,Pave,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,...,80,32,0,0,0,0,700,132000,14,14
2928,20,RL,10010,Pave,Reg,Lvl,AllPub,Inside,Mod,Mitchel,...,240,38,0,0,0,0,0,170000,32,31


We have finished the Features Engineering process. Next step is to choose the columns which could help us to perform the best machine leaning algorithm.

# Features Selection

To start our selection, let us first consider numeric columns which are highly correlated to the column 'SalePrice'. We only keep the columns in which the correlation is larger than 0.4.



In [11]:
num_columns=df.select_dtypes(include = ['integer','float'])
num_columns.head()



,MS SubClass,Lot Area,Overall Qual,Overall Cond,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,...,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,SalePrice,Year_since_built,Year_since_remod
0,20,31770,6,5,112.0,639.0,0.0,441.0,1080.0,1656,...,210,62,0,0,0,0,0,215000,50,50
1,20,11622,5,6,0.0,468.0,144.0,270.0,882.0,896,...,140,0,0,0,120,0,0,105000,49,49
2,20,14267,6,6,108.0,923.0,0.0,406.0,1329.0,1329,...,393,36,0,0,0,0,12500,172000,52,52
3,20,11160,7,5,0.0,1065.0,0.0,1045.0,2110.0,2110,...,0,0,0,0,0,0,0,244000,42,42
4,60,13830,5,5,0.0,791.0,0.0,137.0,928.0,928,...,212,34,0,0,0,0,0,189900,13,12


In [12]:
corr_abs=num_columns.corr()['SalePrice'].abs().sort_values()
corr_abs

BsmtFin SF 2        0.006127
Misc Val            0.019273
3Ssn Porch          0.032268
Bsmt Half Bath      0.035875
Low Qual Fin SF     0.037629
Pool Area           0.068438
MS SubClass         0.085128
Overall Cond        0.101540
Screen Porch        0.112280
Kitchen AbvGr       0.119760
Enclosed Porch      0.128685
Bedroom AbvGr       0.143916
Bsmt Unf SF         0.182751
Lot Area            0.267520
2nd Flr SF          0.269601
Bsmt Full Bath      0.276258
Half Bath           0.284871
Open Porch SF       0.316262
Wood Deck SF        0.328183
BsmtFin SF 1        0.439284
Fireplaces          0.474831
TotRms AbvGrd       0.498574
Mas Vnr Area        0.506983
Year_since_remod    0.534985
Full Bath           0.546118
Year_since_built    0.558979
1st Flr SF          0.635185
Garage Area         0.641425
Total Bsmt SF       0.644012
Garage Cars         0.648361
Gr Liv Area         0.717596
Overall Qual        0.801206
SalePrice           1.000000
Name: SalePrice, dtype: float64

In [13]:
df = df.drop(corr_abs[corr_abs < 0.4].index, axis=1)
df

,MS Zoning,Street,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,...,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Garage Cars,Garage Area,Paved Drive,SalePrice,Year_since_built,Year_since_remod
0,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,...,TA,7,Typ,2,2.0,528.0,P,215000,50,50
1,RH,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,...,TA,5,Typ,0,1.0,730.0,Y,105000,49,49
2,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,...,Gd,6,Typ,0,1.0,312.0,Y,172000,52,52
3,RL,Pave,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,...,Ex,8,Typ,2,2.0,522.0,Y,244000,42,42
4,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,...,TA,6,Typ,1,2.0,482.0,Y,189900,13,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,RL,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,Mitchel,Norm,Norm,...,TA,6,Typ,0,2.0,588.0,Y,142500,22,22
2926,RL,Pave,IR1,Low,AllPub,Inside,Mod,Mitchel,Norm,Norm,...,TA,5,Typ,0,2.0,484.0,Y,131000,23,23
2927,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,...,TA,6,Typ,0,0.0,0.0,Y,132000,14,14
2928,RL,Pave,Reg,Lvl,AllPub,Inside,Mod,Mitchel,Norm,Norm,...,TA,6,Typ,1,2.0,418.0,Y,170000,32,31


Next step is to convert the remaning columns in to categorical data type in order to use our algorithm. But before that, one need to determine which columns needing to be converted and which ones we should remove. 

Basically, all nomial columns are categorical. But we should delete the columns which contains so many unique values. The reason is that when we dummy code these columns, there will be hundred columns added back to the data frame. 

One more observation, there are numeric columns which do not have any meaning to the problem. We also need to convert these columns to categorical ones.

In [14]:
# list of all columns at the beginning that are categorical
nominal_features = ['PID', 'MS SubClass', 'MS Zoning', 'Street', 'Alley', 'Land Contour', 'Lot Config', 'Neighborhood', 
                    'Condition 1', 'Condition 2', 'Bldg Type', 'House Style', 'Roof Style', 'Roof Matl', 'Exterior 1st', 
                    'Exterior 2nd', 'Mas Vnr Type', 'Foundation', 'Heating', 'Central Air', 'Garage Type', 
                    'Misc Feature', 'Sale Type', 'Sale Condition']

# compare to the clean dataframe after features engineering
remaining_cols=[]
for col in nominal_features:
    if col in df.columns:
        remaining_cols.append(col)
remaining_cols        

['MS Zoning',
 'Street',
 'Land Contour',
 'Lot Config',
 'Neighborhood',
 'Condition 1',
 'Condition 2',
 'Bldg Type',
 'House Style',
 'Roof Style',
 'Roof Matl',
 'Exterior 1st',
 'Exterior 2nd',
 'Foundation',
 'Heating',
 'Central Air']

In [15]:
# only keep the features with at most 10 unique values
unique_values_counts = df[remaining_cols].apply(lambda col: len(col.value_counts())).sort_values()

drop_cols = unique_values_counts[unique_values_counts > 10].index
df = df.drop(drop_cols, axis=1)
df

,MS Zoning,Street,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Condition 1,Condition 2,Bldg Type,...,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Garage Cars,Garage Area,Paved Drive,SalePrice,Year_since_built,Year_since_remod
0,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Norm,Norm,1Fam,...,TA,7,Typ,2,2.0,528.0,P,215000,50,50
1,RH,Pave,Reg,Lvl,AllPub,Inside,Gtl,Feedr,Norm,1Fam,...,TA,5,Typ,0,1.0,730.0,Y,105000,49,49
2,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Norm,Norm,1Fam,...,Gd,6,Typ,0,1.0,312.0,Y,172000,52,52
3,RL,Pave,Reg,Lvl,AllPub,Corner,Gtl,Norm,Norm,1Fam,...,Ex,8,Typ,2,2.0,522.0,Y,244000,42,42
4,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,Norm,Norm,1Fam,...,TA,6,Typ,1,2.0,482.0,Y,189900,13,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,RL,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,Norm,Norm,1Fam,...,TA,6,Typ,0,2.0,588.0,Y,142500,22,22
2926,RL,Pave,IR1,Low,AllPub,Inside,Mod,Norm,Norm,1Fam,...,TA,5,Typ,0,2.0,484.0,Y,131000,23,23
2927,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,1Fam,...,TA,6,Typ,0,0.0,0.0,Y,132000,14,14
2928,RL,Pave,Reg,Lvl,AllPub,Inside,Mod,Norm,Norm,1Fam,...,TA,6,Typ,1,2.0,418.0,Y,170000,32,31


In [16]:
# convert remaning text columns to categorical data type
text_cols = df.select_dtypes(include=['object'])
for col in text_cols:
    df[col] = df[col].astype('category')
    
# Create dummy columns and then add back to our dataframe!
df = pd.concat([df, pd.get_dummies(df.select_dtypes(include=['category']))], axis=1).drop(text_cols,axis=1) 
df

,Overall Qual,Mas Vnr Area,BsmtFin SF 1,Total Bsmt SF,1st Flr SF,Gr Liv Area,Full Bath,TotRms AbvGrd,Fireplaces,Garage Cars,...,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sal,Functional_Sev,Functional_Typ,Paved Drive_N,Paved Drive_P,Paved Drive_Y
0,6,112.0,639.0,1080.0,1656,1656,1,7,2,2.0,...,0,0,0,0,0,0,1,0,1,0
1,5,0.0,468.0,882.0,896,896,1,5,0,1.0,...,0,0,0,0,0,0,1,0,0,1
2,6,108.0,923.0,1329.0,1329,1329,1,6,0,1.0,...,0,0,0,0,0,0,1,0,0,1
3,7,0.0,1065.0,2110.0,2110,2110,2,8,2,2.0,...,0,0,0,0,0,0,1,0,0,1
4,5,0.0,791.0,928.0,928,1629,2,6,1,2.0,...,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,6,0.0,819.0,1003.0,1003,1003,1,6,0,2.0,...,0,0,0,0,0,0,1,0,0,1
2926,5,0.0,301.0,864.0,902,902,1,5,0,2.0,...,0,0,0,0,0,0,1,0,0,1
2927,5,0.0,337.0,912.0,970,970,1,6,0,0.0,...,0,0,0,0,0,0,1,0,0,1
2928,5,0.0,1071.0,1389.0,1389,1389,1,6,1,2.0,...,0,0,0,0,0,0,1,0,0,1


We have done features selection. It is time to start our linear regression algorithm.

# Linear Regression Algorithm

For more convenience, we create a function called train_and_test() to perform our algorithm.

In [17]:
def train_and_test(df, k=0):
    numeric_df = df.select_dtypes(include=['integer', 'float'])
    features = numeric_df.columns.drop("SalePrice")
    lr = linear_model.LinearRegression()
    
    if k == 0:# hold out validation
        train = df[:1460] # 50% of the rows
        test = df[1460:]

        lr.fit(train[features], train["SalePrice"])
        predictions = lr.predict(test[features])
        mse = mean_squared_error(test["SalePrice"], predictions)
        rmse = np.sqrt(mse)

        return rmse
    
    if k == 1: # simple cross validation
        # Randomize rows (frac=1) from df 
        shuffled_df = df.sample(frac=1, )
        train = df[:1460]
        test = df[1460:]
        
        lr.fit(train[features], train["SalePrice"])
        predictions_one = lr.predict(test[features])        
        
        mse_one = mean_squared_error(test["SalePrice"], predictions_one)
        rmse_one = np.sqrt(mse_one)
        lr.fit(test[features], test["SalePrice"])
        predictions_two = lr.predict(train[features])        
       
        mse_two = mean_squared_error(train["SalePrice"], predictions_two)
        rmse_two = np.sqrt(mse_two)
        
        avg_rmse = np.mean([rmse_one, rmse_two])
        print(rmse_one)
        print(rmse_two)
        return avg_rmse
    else: # k-fold cross validation
        kf = KFold(n_splits=k, shuffle=True)
        rmse_values = []
        for train_index, test_index, in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            lr.fit(train[features], train["SalePrice"])
            predictions = lr.predict(test[features])
            mse = mean_squared_error(test["SalePrice"], predictions)
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
        print(rmse_values)
        avg_rmse = np.mean(rmse_values)
        return avg_rmse

In [18]:
# Print the rmse values for some first k values to see which one performing the best
for i in range(20):
    rmse = train_and_test(df, k=i)
    print('The average rmse when k={}: '.format(i), rmse)
    print('\n')


The average rmse when k=0:  33367.28718340389


33367.28718340389
27155.138484825173
The average rmse when k=1:  30261.21283411453


[27419.075346960486, 31780.821154758374]
The average rmse when k=2:  29599.94825085943


[27587.983037160266, 33475.07193543799, 27083.473997127083]
The average rmse when k=3:  29382.176323241776


[27006.891781221006, 27013.99261813576, 36736.46761667886, 25734.367393527235]
The average rmse when k=4:  29122.929852390716


[26779.609407273838, 40018.57809359882, 24780.16044807409, 27599.468764869664, 24115.861810635564]
The average rmse when k=5:  28658.7357048904


[28273.714317262606, 39231.60855754468, 26949.573020213036, 28736.222729011562, 24727.348125746743, 25274.308041716577]
The average rmse when k=6:  28865.462465249202


[27185.958654404974, 40901.46413546588, 24359.487503644774, 33167.4873764958, 23368.74347728876, 24882.130157307372, 28266.64591522677]
The average rmse when k=7:  28875.98817426205


[26469.14015150758, 28076.541691126382, 26